# Model Gallery

**TODO: Write me!**

This is an overview of the Gammapy built-in models in `~gammapy.modeling.models`.

Note that there is a separate tutorial [modeling](modeling.ipynb) that explains about `~gammapy.modeling`,
the Gammapy modeling and fitting framework. You have to read that to learn how to work with models.

Topics covered here:

- How to create spatial, spectral and temporal models.
- How to create 3D sky models and other compound models.
- How to use the model registries to list all available models or serialise models.
- How to work with user defined models for simulations and fitting.
- How to serialize/read and deserialize/write models.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import Angle
from gammapy.maps import Map, WcsGeom
import gammapy.modeling.models as gm
from gammapy.modeling import Parameter
from gammapy.modeling.models import (
    SpectralModel,
    PowerLawSpectralModel,
    SkyModels,
    SkyModel,
)

## Spatial models

In [ ]:
m_geom = WcsGeom.create(
    binsz=0.01, width=(5, 5), skydir=(2, 2), coordsys="GAL", proj="AIT"
)
phi = Angle("30 deg")
gaussian = gm.GaussianSpatialModel(
    lon_0="2 deg",
    lat_0="2 deg",
    sigma="1 deg",
    e=0.7,
    phi=phi,
    frame="galactic",
)

coords = m_geom.get_coord()
vals = gaussian(coords.lon, coords.lat)
skymap = Map.from_geom(m_geom, data=vals.value)

_, ax, _ = skymap.smooth("0.05 deg").plot()

transform = ax.get_transform("galactic")
ax.scatter(2, 2, transform=transform, s=20, edgecolor="red", facecolor="red")
ax.text(1.5, 1.85, r"$(l_0, b_0)$", transform=transform, ha="center")
ax.plot(
    [2, 2 + np.sin(phi)], [2, 2 + np.cos(phi)], color="r", transform=transform
)
ax.vlines(x=2, color="r", linestyle="--", transform=transform, ymin=-5, ymax=5)
ax.text(2.25, 2.45, r"$\phi$", transform=transform)
ax.contour(skymap.data, cmap="coolwarm", levels=10, alpha=0.6);

In [ ]:
disk = gm.DiskSpatialModel(
    lon_0="2 deg",
    lat_0="2 deg",
    r_0="1 deg",
    e=0.8,
    phi="30 deg",
    frame="galactic",
)

m_geom = WcsGeom.create(
    binsz=0.01, width=(3, 3), skydir=(2, 2), coordsys="GAL", proj="AIT"
)
coords = m_geom.get_coord()
vals = disk(coords.lon, coords.lat)
skymap = Map.from_geom(m_geom, data=vals.value)

_, ax, _ = skymap.smooth("0.05 deg").plot()

transform = ax.get_transform("galactic")
ax.scatter(2, 2, transform=transform, s=20, edgecolor="red", facecolor="red")
ax.text(1.7, 1.85, r"$(l_0, b_0)$", transform=transform, ha="center")
ax.plot(
    [2, 2 + np.sin(np.pi / 6)],
    [2, 2 + np.cos(np.pi / 6)],
    color="r",
    transform=transform,
)
ax.vlines(x=2, color="r", linestyle="--", transform=transform, ymin=0, ymax=5)
ax.text(2.15, 2.3, r"$\phi$", transform=transform);

## Spectral models

In [ ]:
energy_range = [0.1, 100] * u.TeV

pwl = gm.PowerLawSpectralModel()
pwl.plot(energy_range)

pwl2 = gm.PowerLaw2SpectralModel()
pwl2.plot(energy_range)

ecpl = gm.ExpCutoffPowerLawSpectralModel()
ecpl.plot(energy_range)

ecpl_3fgl = gm.ExpCutoffPowerLaw3FGLSpectralModel()
ecpl_3fgl.plot(energy_range)

secpl_3fgl = gm.SuperExpCutoffPowerLaw3FGLSpectralModel()
secpl_3fgl.plot(energy_range)

secpl_4fgl = gm.SuperExpCutoffPowerLaw4FGLSpectralModel()
secpl_4fgl.plot(energy_range)

log_parabola = gm.LogParabolaSpectralModel()
log_parabola.plot(energy_range)

plt.ylim(1e-18, 1e-10);

In [ ]:
# Create and plot EBL absorption models for a redshift of 0.5

redshift = 0.5
dominguez = gm.Absorption.read_builtin("dominguez").table_model(redshift)
franceschini = gm.Absorption.read_builtin("franceschini").table_model(redshift)
finke = gm.Absorption.read_builtin("finke").table_model(redshift)

plt.figure()
energy_range = [0.08, 3] * u.TeV
opts = dict(energy_range=energy_range, energy_unit="TeV", flux_unit="")
franceschini.plot(label="Franceschini 2008", **opts)
finke.plot(label="Finke 2010", **opts)
dominguez.plot(label="Dominguez 2011", **opts)

plt.ylabel(r"Absorption coefficient [$\exp{(-\tau(E))}$]")
plt.xlim(energy_range.value)
plt.ylim(1e-4, 2)
plt.title(f"EBL models (z={redshift})")
plt.grid(which="both")
plt.legend(loc="best");

In [ ]:
# Create and plot a spectral model that convolves an `ExpCutoffPowerLawSpectralModel` electron distribution
# with an `InverseCompton` radiative model, in the presence of multiple seed photon fields.

import naima

particle_distribution = naima.models.ExponentialCutoffPowerLaw(
    1e30 / u.eV, 10 * u.TeV, 3.0, 30 * u.TeV
)
radiative_model = naima.radiative.InverseCompton(
    particle_distribution,
    seed_photon_fields=["CMB", ["FIR", 26.5 * u.K, 0.415 * u.eV / u.cm ** 3]],
    Eemin=100 * u.GeV,
)

model = gm.NaimaSpectralModel(radiative_model, distance=1.5 * u.kpc)

opts = {
    "energy_range": [10 * u.GeV, 80 * u.TeV],
    "energy_power": 2,
    "flux_unit": "erg-1 cm-2 s-1",
}

# Plot the total inverse Compton emission
model.plot(label="IC (total)", **opts)

# Plot the separate contributions from each seed photon field
for seed, ls in zip(["CMB", "FIR"], ["-", "--"]):
    model = gm.NaimaSpectralModel(
        radiative_model, seed=seed, distance=1.5 * u.kpc
    )
    model.plot(label=f"IC ({seed})", ls=ls, color="gray", **opts)

plt.legend(loc="best");

In [ ]:
gaussian = gm.GaussianSpectralModel(mean="1 TeV")
gaussian.plot(energy_range);


Many spectral models in gammapy are subclasses of ~gammapy.modeling.models.SpectralModel. The list of available models is shown below.


In [ ]:
SpectralModel.__subclasses__()

## Custom models

In order to add a user defined spectral model you have to create a SpectralModel subclass.
This new model class should include:

- a tag used for serialization (it can be the same as the class name)
- an instantiation of each Parameter with their unit, default values and frozen status
- the evaluate function where the mathematical expression for the model is defined.

As an example we will use a PowerLawSpectralModel plus a Gaussian (with fixed width).
First we define the new custom model class that we name `PLG`:

In [ ]:
class PLG(SpectralModel):
    tag = "PLG"
    index = Parameter("index", 2, min=0)
    amplitude = Parameter("amplitude", "1e-12 cm-2 s-1 TeV-1", min=0)
    reference = Parameter("reference", "1 TeV", frozen=True)
    mean = Parameter("mean", "1 TeV", min=0)
    width = Parameter("width", "0.1 TeV", min=0, frozen=True)

    @staticmethod
    def evaluate(energy, index, amplitude, reference, mean, width):
        pwl = PowerLawSpectralModel.evaluate(
            energy=energy,
            index=index,
            amplitude=amplitude,
            reference=reference,
        )
        gauss = amplitude * np.exp(-((energy - mean) ** 2) / (2 * width ** 2))
        return pwl + gauss

then we add it to the spectral model registry so it can be used for fitting and serialization:

In [ ]:
from gammapy.modeling.models import SPECTRAL_MODELS

SPECTRAL_MODELS.append(PLG)

In [ ]:
custom_model = PLG(
    index=2,
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    reference=1 * u.TeV,
    mean=5 * u.TeV,
    width=0.2 * u.TeV,
)
print(custom_model)

In [ ]:
energy_range = [1, 10] * u.TeV
custom_model.plot(energy_range=energy_range);

Note that gammapy assumes that all SpectralModel evaluate functions return a flux in unit of "cm-2 s-1 TeV-1" (or equivalent dimensions).

Similarly you can also create custom spatial models and add them to the `SPATIAL_MODELS` registry. In that case gammapy assumes that the evaluate function return a normalized quantity in "sr-1" such as the model integral over the whole sky is one.

Once your custom models are defined and added to their model resgistry they can be serialized like the built-in models, as shown at the end of this tutorial.

## 3D model and models list

A source can be modeled by a combination of a spatial and a spectral model using a `SkyModel`. 
For example we can use the disk and exponential cut-off power-law models defined previously to create a new source model.

In [ ]:
model1 = SkyModel(spectral_model=ecpl, spatial_model=disk)
print(model1)

Note that for convenience the spatial model component is optionnal. Here we create a source model using only the  power-law model defined previously:


In [ ]:
model2 = SkyModel(pwl)
print(model2)

Then the global model of an analysis can be define by combining several `SkyModel` into a `SkyModels`.



In [ ]:
models = SkyModels([model1, model2])

This model object will be assigned to `Dataset` or `Datasets` together with the data to be fitted as explained in other analysis tutotials (see for example the [modeling](modeling.ipynb) notebook).


## Serialization

The list of models contained in a `SkyModels` object can be exported/imported using yaml configuration files.



In [ ]:
models_yaml = models.to_yaml()
print(models_yaml)

The parameters keys min/max/frozen are optionnals, so you can prepare shorter files.

If you want to write this list of models to disk and read it back later you can use:

In [ ]:
models.write("models.yaml", overwrite=True)
models_read = SkyModels.read("models.yaml")

Additionnaly the models can exported and imported togeter with the data using the `Datasets.read()` and `Datasets.write()` methods as shown in the [analysis_mwl](analysis_mwl) notebook.